# Disease Mapping: John Snow and Cholera

## Introduction

*What's So Special about Spatial?*

*Everything is related to everything else, but near things are more related than distant things.* Waldo tobler

In [ ]:
# Install the required packages

# We are going to use geopandas, an open source project to make working with geospatial data in python easier.
# --> https://geopandas.org/

!pip install geopandas
!pip install descartes

# Rasterio reads and writes raster formats
# --> https://rasterio.readthedocs.io/en/latest/

!pip install rasterio

![](https://www.fastprint.co.uk/Assets/User/17-1-vector-vs-raster.jpg)

*Can you tell the difference between these two data representations?*

## Recreating the John Snow Map

In [ ]:
import geopandas
import descartes

In [ ]:
# Read data files
pumps = geopandas.read_file("https://data-deposit.s3.eu-central-1.amazonaws.com/snow-walk/Pumps.shp")
Cholera_Deaths = geopandas.read_file("https://data-deposit.s3.eu-central-1.amazonaws.com/snow-walk/Cholera_Deaths.shp")

In [ ]:
# Check head
pumps.head()

In [ ]:
# Check CRS
pumps.crs

In [ ]:
pumps.shape

In [ ]:
pumps.plot();

In [ ]:
# Now let's look at the colera_deaths dataset
Cholera_Deaths.head()

In [ ]:
Cholera_Deaths.plot(marker='o', color='red', markersize=5);

In [ ]:
# Let's have a look at a Snow's original map
import rasterio
from rasterio.plot import show

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(num=None, figsize=(10, 10), dpi=80, facecolor='w', edgecolor='k')
# matplotlib colors: https://matplotlib.org/stable/gallery/color/named_colors.html#base-colors

src = rasterio.open("https://data-deposit.s3.eu-central-1.amazonaws.com/snow-walk/SnowMap.tif")
plt.imshow(src.read(1), cmap='pink')
plt.show()




*Do you notice something different about this map?*

In [ ]:
# Let's overlay everything in a single map

fig, ax = plt.subplots(figsize=(40, 20))
ax.set_aspect('equal')

pumps.plot(ax=ax, marker='o', color='blue', markersize=100)
Cholera_Deaths.plot(ax=ax, marker='o', color='red', markersize=10)
show(src, cmap='Greys', ax=ax)

plt.show()



Choosing a colormap:
https://matplotlib.org/tutorials/colors/colormaps.html

We want to relate the cholera deaths with the water pumps, so let's check all the deaths in the surroundings of a water pump.
*How can we do this?*

## Heatmap

In [ ]:
Cholera_Deaths['x'] = Cholera_Deaths['geometry'].x
Cholera_Deaths['y'] = Cholera_Deaths['geometry'].y

In [ ]:
Cholera_Deaths

In [ ]:
!pip install seaborn
import seaborn as sns

In [ ]:
# Plot a heatmap, using a Kernel Density estimation (KDE), to estimate the probability density function
sns.kdeplot(x=Cholera_Deaths['x'],
            y=Cholera_Deaths['y'],
            color='r', shade=True,
            cmap="Reds", shade_lowest=False)

In [ ]:
# Now, let's overlay the heatmap to the map we previously done.
# Note that you can use the alpha channel, to tweak the transparency!
fig, ax = plt.subplots(figsize=(40, 20))
ax.set_aspect('equal')
pumps.plot(ax=ax, marker='o', color='blue', markersize=100)
sns.kdeplot(x=Cholera_Deaths['x'],
            y=Cholera_Deaths['y'],
            color='r', shade=True, Label='Cholera Deaths',
            cmap="Reds", shade_lowest=False, alpha  = 0.7)
show(src, cmap='Greys', ax=ax)
plt.show();

## Raster

In [ ]:
#Let's have a look at raster data with more detail

src = rasterio.open("https://data-deposit.s3.eu-central-1.amazonaws.com/snow-walk/SnowDSM50CM.tif")
array = src.read(1)

array.shape



In [ ]:
array

In [ ]:
from matplotlib.pyplot import figure
figure(num=None, figsize=(25, 25), dpi=80, facecolor='w', edgecolor='k')
plt.imshow(src.read(1), cmap='viridis')
plt.show()


## Buffers

A buffer in GIS is a zone around a map feature measured in units of distance or time. A buffer is useful for **proximity analysis**.

![](https://desktop.arcgis.com/en/arcmap/10.3/tools/analysis-toolbox/GUID-267CF0D1-DB92-456F-A8FE-F819981F5467-web.png)

In [ ]:
# Create some buffers

buffers=pumps.buffer(50)
fig, ax = plt.subplots(figsize=(40, 20))
ax.set_aspect('equal')

buffers.plot(ax=ax, marker='o', color='yellow', alpha=0.5, markersize=100)
pumps.plot(ax=ax, marker='o', color='blue', markersize=100)

Cholera_Deaths.plot(ax=ax, marker='o', color='red', markersize=10)
show(src, cmap='Greys', ax=ax)

plt.show()


*Try to change the radius of the buffer. What happens?*

In [ ]:
buffers

We can see that many points lie within a particular buffer, but now we want to see ony the points that fall whithin the buffer. For that we need to transform the buffers into a data frame.

In [ ]:
pumps2 = pumps
pumps2['geometry']=pumps.buffer(50)
pumps2.plot()


## Spatial Overlays

In [ ]:
!pkexec apt-get install -qq curl g++ make


In [ ]:
!curl -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz

In [ ]:
import os
os.chdir('spatialindex-src-1.8.5')

In [ ]:
!./configure

In [36]:
!make

libtool: compile:  g++ -DPACKAGE_NAME=\"spatialindex\" -DPACKAGE_TARNAME=\"spatialindex-src\" -DPACKAGE_VERSION=\"1.8.5\" "-DPACKAGE_STRING=\"spatialindex 1.8.5\"" -DPACKAGE_BUGREPORT=\"mhadji@gmail.com\" -DPACKAGE_URL=\"\" -DSTDC_HEADERS=1 -DHAVE_SYS_TYPES_H=1 -DHAVE_SYS_STAT_H=1 -DHAVE_STDLIB_H=1 -DHAVE_STRING_H=1 -DHAVE_MEMORY_H=1 -DHAVE_STRINGS_H=1 -DHAVE_INTTYPES_H=1 -DHAVE_STDINT_H=1 -DHAVE_UNISTD_H=1 -DHAVE_DLFCN_H=1 -DLT_OBJDIR=\".libs/\" -DPACKAGE=\"spatialindex-src\" -DVERSION=\"1.8.5\" -DHAVE_FCNTL_H=1 -DHAVE_UNISTD_H=1 -DHAVE_SYS_TYPES_H=1 -DHAVE_SYS_STAT_H=1 -DHAVE_PTHREAD_H=1 -DHAVE_SYS_RESOURCE_H=1 -DHAVE_SYS_TIME_H=1 -DHAVE_STDINT_H=1 -DHAVE_FEATURES_H=1 -DHAVE_GETTIMEOFDAY=1 -DHAVE_MEMSET=1 -DHAVE_MEMCPY=1 -DHAVE_BCOPY=1 -DHAVE_SRAND48=1 -I. -I../../include -Wall -Wno-long-long -pedantic -std=c++98 -O2 -DNDEBUG -MT TPRTree.lo -MD -MP -MF .deps/TPRTree.Tpo -c TPRTree.cc -o TPRTree.o >/dev/null 2>&1
depbase=`echo Statistics.lo | sed 's|[^/]*$|.deps/&|;s|\.lo$||'`;\
/bin/

In [37]:
!make install

Making install in src
make[1]: Entering directory '/home/joana/git/snow-walk-public/snow-walk-public/spatialindex-src-1.8.5/spatialindex-src-1.8.5/src'
Making install in storagemanager
make[2]: Entering directory '/home/joana/git/snow-walk-public/snow-walk-public/spatialindex-src-1.8.5/spatialindex-src-1.8.5/src/storagemanager'
make[3]: Entering directory '/home/joana/git/snow-walk-public/snow-walk-public/spatialindex-src-1.8.5/spatialindex-src-1.8.5/src/storagemanager'
make[3]: Nothing to be done for 'install-exec-am'.
make[3]: Nothing to be done for 'install-data-am'.
make[3]: Leaving directory '/home/joana/git/snow-walk-public/snow-walk-public/spatialindex-src-1.8.5/spatialindex-src-1.8.5/src/storagemanager'
make[2]: Leaving directory '/home/joana/git/snow-walk-public/snow-walk-public/spatialindex-src-1.8.5/spatialindex-src-1.8.5/src/storagemanager'
Making install in spatialindex
make[2]: Entering directory '/home/joana/git/snow-walk-public/snow-walk-public/spatialindex-src-1.8.5/sp

In [38]:
!ldconfig -p | grep spatialindex

	libspatialindex_c.so.6 (libc6,x86-64) => /lib/x86_64-linux-gnu/libspatialindex_c.so.6
	libspatialindex_c.so (libc6,x86-64) => /lib/x86_64-linux-gnu/libspatialindex_c.so
	libspatialindex.so.6 (libc6,x86-64) => /lib/x86_64-linux-gnu/libspatialindex.so.6
	libspatialindex.so (libc6,x86-64) => /lib/x86_64-linux-gnu/libspatialindex.so


In [39]:
! apt-get install -y libspatialindex-dev

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [40]:
# We are going to overlay the deaths with the new layer, using the intersect function

# Install the required packages
!pip install rtree
!pip install pygeos

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [34 lines of output]
      <string>:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
      /tmp/pip-install-8img504c/pygeos_0b5fd00e91c84f2fa5c9463aa67d8071/versioneer.py:421: SyntaxWarning: invalid escape sequence '\s'
        LONG_VERSION_PY['git'] = '''
      Compiling pygeos/_geometry.pyx because it changed.
      Compiling pygeos/_geos.pyx because it changed.
      [1/2] Cythonizing pygeos/_geometry.pyx
      [2/2] Cythonizing pygeos/_geos.pyx
      Traceback (most recent call last):
        Fil

![](https://geopandas.org/_images/overlay_operations.png)

We want to find all the cholera deaths, which **intersect** the pumps buffer.

In [41]:
from rtree import index
from rtree.index import Rtree

In [42]:
intersect = geopandas.overlay(Cholera_Deaths, pumps2, how='intersection')

In [ ]:
intersect.plot()

In [ ]:
intersect

Now prepare the results for Kaggle

In [ ]:
# 1 - Groupby Id_2, suming the reported cases.

In [ ]:
# 2 - Check the results

In [ ]:
# 3 - drop Id_1 column (we only need the ID of the pumps and the count)

In [ ]:
# 4 - export the results


# Now remember to add the other pumps which contained zero cases!

In [ ]:
# 5 - Submit the results to the Kaggle competition:
# https://www.kaggle.com/competitions/snow-cholera-analysis

In [ ]:
fig, ax = plt.subplots(figsize=(40, 20))
ax.set_aspect('equal')
ax.set_title("Colera Deaths within 50m of a pump", fontsize= 30);
pumps.plot(ax=ax, alpha=0.2, marker='o', color='blue', markersize=100)

intersect.plot(ax=ax, marker='P', color='black', markersize=50)

src = rasterio.open("https://data-deposit.s3.eu-central-1.amazonaws.com/snow-walk/SnowDSM50CM.tif")
show(src, cmap='Greys', ax=ax)

plt.show();


In [ ]:
# Save this, to put it into a report :-)
fig.savefig('plot.png')

In [ ]:
# We could add a context tile map
!pip install contextily
import contextily as ctx

https://leaflet-extras.github.io/leaflet-providers/preview/

In [ ]:
# Reproject the layers to show them on the map
intersect = intersect.to_crs(epsg=3857)
pumps = pumps.to_crs(epsg=3857)

fig, ax = plt.subplots(figsize=(20, 10))
ax.set_aspect('equal')
ax.set_title("Colera Deaths within 50m of a Water Pump", fontsize= 30);
pumps.plot(ax=ax, alpha=0.2, marker='o', color='blue', markersize=100)

intersect.plot(ax=ax, marker='P', color='red', markersize=50)

# Add your API key to the desired style from contextily providers
# provider = ctx.providers.Stadia.StamenWatercolor(api_key="YOUR-API-KEY")

# Update the provider URL to include your API key
# provider["url"] = provider["url"] + "?api_key={api_key}"

#ctx.add_basemap(ax, source=provider)

ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

plt.show();

# Use stadia maps with contextily https://docs.stadiamaps.com/tutorials/getting-started-with-geopandas-contextily/

## Voronoi Diagrams

The buffers are useful to tell us which cases are within certain absolute distance of a water pump, but they don't tell us to which water pump, all the other cases outside the buffers are going. They certainly muts be getting their water somewhere!

A Voronoi diagram is a visualisation of an area partitioned into regions that minimise the distance to given point locations. These diagrams are also known variously as Voronoi tessellations, Dirichlet tessellation and Thiessen polygons. An example is shown below.

![](https://raw.githubusercontent.com/jamesdamillington/john-snow/deeced0847898db1f7bff2f62565235833b9f449/data/img/Euclidean_Voronoi_diagram.png)

Voronoi diagrams are constructed using a similar method to buffers around points (see Operations notebook), but ensure that there are no overlaps between polygons. Voronoi digrams are also useful for thinking about spatial neighbourhoods. There are numerous processes Voronoi diagrams have been used to investigate.

In [ ]:
!pip install shapely

In [ ]:
!pip install libpysal
from libpysal.cg.voronoi import voronoi, voronoi_frames
import numpy as np

In [ ]:
pumps = geopandas.read_file("https://data-deposit.s3.eu-central-1.amazonaws.com/snow-walk/Pumps.shp")
blocks = geopandas.read_file('https://data-deposit.s3.eu-central-1.amazonaws.com/snow-walk/polys.shp')

In [ ]:
#extract the geometry into a numerical field
pumps['x'] = pumps['geometry'].x
pumps['y'] = pumps['geometry'].y

In [ ]:
pumps

In [ ]:
#stack arrays in sequence vertically (row wise)
points = np.vstack([pumps['x'], pumps['y']]).T

In [ ]:
points

In [ ]:
regions_df_noclip, vertices_df_noclip = voronoi_frames(points, clip='none')

In [ ]:
regions_df_noclip

In [ ]:
vertices_df_noclip

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))

ax.set_xlim(blocks.total_bounds[0],blocks.total_bounds[2])   #use bounding box of blocks
ax.set_ylim(blocks.total_bounds[1],blocks.total_bounds[3])   #use bounding box of blocks

Cholera_Deaths.plot(ax=ax, marker='o', color='blue', markersize=10)
blocks.plot(ax=ax, facecolor='0.9', linewidth=0)
regions_df_noclip.plot(ax=ax, color='lightblue',edgecolor='black', alpha=0.3)
vertices_df_noclip.plot(ax=ax, color='red')

In [ ]:
regions_df_noclip['ID'] = range(0, len(regions_df_noclip))

In [ ]:
regions_df_noclip = regions_df_noclip.set_geometry('geometry')

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
ax.set_xlim(blocks.total_bounds[0],blocks.total_bounds[2])   #use bounding box of blocks
ax.set_ylim(blocks.total_bounds[1],blocks.total_bounds[3])   #use bounding box of blocks
blocks.plot(ax=ax, facecolor='0.9', linewidth=0)

Cholera_Deaths.plot(ax=ax, marker='o', color='black', markersize=10)
#use column here with a colourmap
regions_df_noclip.plot(ax=ax, column='ID', cmap='Set1', edgecolor='black', alpha=0.3)
vertices_df_noclip.plot(ax=ax, color='red')

## References

V. Olaya. Introduction to GIS. CreateSpace Independent Publishing Platform, 2018.
https://volaya.github.io/gis-book/en/index.html

### Spatial reference systems

https://spatialreference.org/

https://store.usgs.gov/assets/mod/storefiles/PDF/16573.pdf

### John Snow & Classics

http://blog.rtwilson.com/john-snows-cholera-data-in-more-formats/

https://www.theguardian.com/news/datablog/2013/mar/15/john-snow-cholera-map#data

### Thematic Cartography

www.geog.ucsb.edu/~kclarke/Geography183/Lecture03.pdf

http://www.gitta.info/ThematicCart/en/html/index.html

https://www.axismaps.com/guide/

https://www.axismaps.com/guide/univariate/choropleth/

https://www.axismaps.com/guide/data/data-classification/
